In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
def resampling (df):
  

    # Convert the timestamp column to datetime format
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ns')

    # Set the timestamp column as the DataFrame index
    df = df.set_index('timestamp')

    # Resample the DataFrame to a frequency of 100 Hz
    df_resampled = df.resample('10L').mean().interpolate()  # '10L' represents 10 milliseconds (100 Hz)

    # Print the resampled DataFrame
    return df_resampled


In [3]:

root_folder = "IDNet_dataset (1)"
gyro_final = pd.DataFrame(columns=[ 'gyroscope_x_data', 'gyroscope_y_data','gyroscope_z_data','name'])
acc_final = pd.DataFrame(columns=[ 'linearaccelerometer_x_data', 'linearaccelerometer_y_data','linearaccelerometer_z_data','name'])

# Get a list of all subdirectories in the root folder
subdirectories = [f for f in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, f))]


# Iterate over the subdirectories
for directory in subdirectories:
    if directory.endswith("_w002"):  # Filter based on folder name
        folder_path = os.path.join(root_folder, directory)
        gyro_file = os.path.join(folder_path, directory + "_gyroscope.log")
        accel_file = os.path.join(folder_path, directory + "_linearaccelerometer.log")

        # Check if both log files exist in the current directory
        if os.path.isfile(gyro_file) and os.path.isfile(accel_file):
            # Read the log files into pandas DataFrames
            gyro_data = pd.read_csv(gyro_file, sep='\t')
            accel_data = pd.read_csv(accel_file, sep='\t')
            
            
            accel_data = accel_data.rename(columns={'linearaccelerometer_timestamp': 'timestamp'})
            gyro_data = gyro_data.rename(columns={'gyroscope_timestamp': 'timestamp'})
            
            
            
            acc_resampled = resampling(accel_data)
            gyro_resampled = resampling(gyro_data)
           
            gyro_resampled['name']= directory[1:4]
            acc_resampled['name']= directory[1:4]

            # Concatenate the dataframes
            gyro_final = pd.concat([gyro_final, gyro_resampled], axis=0)
            acc_final = pd.concat([acc_final, acc_resampled], axis=0)



In [4]:
# Define a dictionary with old column names as keys and new column names as values
new_column_names_gyro = { 'gyroscope_x_data': 'GYx', 'gyroscope_y_data': 'GYy','gyroscope_z_data': 'GYz'}
new_column_names_acc = { 'linearaccelerometer_x_data': 'LAx', 'linearaccelerometer_y_data': 'LAy', 'linearaccelerometer_z_data': 'LAz'}
# Rename the columns
gyro_final = gyro_final.rename(columns=new_column_names_gyro)            
acc_final = acc_final.rename(columns = new_column_names_acc)

In [5]:
gyro_final = gyro_final.reset_index()
gyro_final = gyro_final.rename(columns={'index': 'timestamp'})


acc_final = acc_final.reset_index()
acc_final = acc_final.rename(columns={'index': 'timestamp'})


In [6]:
gyro_final['timestamp'] = gyro_final['timestamp'].astype(np.int64)
acc_final['timestamp'] = acc_final['timestamp'].astype(np.int64)

In [7]:
acc_final

,timestamp,LAx,LAy,LAz,name
0,924313410000000,1.242813,-0.160181,-0.900440,001
1,924313420000000,1.117246,-0.187921,-1.529893,001
2,924313430000000,0.912846,-0.192566,-1.570929,001
3,924313440000000,0.874858,-0.227208,-1.453511,001
4,924313450000000,0.834022,-0.282976,-1.292671,001
...,...,...,...,...,...
2055045,1368327120000000,-9.502644,-7.178468,4.246279,050
2055046,1368327130000000,-9.443804,-7.698220,3.059675,050
2055047,1368327140000000,-8.345459,-6.805815,1.853457,050
2055048,1368327150000000,-7.580540,-5.629017,1.559257,050


In [8]:
gyro_final

,timestamp,GYx,GYy,GYz,name
0,924313410000000,0.194931,-0.004242,0.163818,001
1,924313420000000,0.236458,0.076378,0.133293,001
2,924313430000000,0.226074,0.012253,0.108246,001
3,924313440000000,0.139336,-0.050667,0.085030,001
4,924313450000000,-0.013374,-0.109314,0.078308,001
...,...,...,...,...,...
2055103,1368327120000000,-2.834417,-0.120428,-0.987857,050
2055104,1368327130000000,-2.638940,0.561997,-0.947715,050
2055105,1368327140000000,-2.483606,1.225222,-0.954696,050
2055106,1368327150000000,-2.429500,1.769765,-1.003565,050


In [22]:
names = acc_final['name'].unique()
names

array(['001', '002', '003', '004', '005', '006', '007', '008', '009',
       '010', '011', '012', '013', '014', '015', '016', '017', '018',
       '019', '020', '021', '022', '023', '024', '025', '026', '027',
       '028', '029', '030', '031', '032', '033', '034', '035', '036',
       '037', '038', '039', '040', '041', '042', '043', '044', '045',
       '046', '047', '048', '049', '050'], dtype=object)

In [25]:
merged_final = pd.DataFrame(columns=["timestamp",'LAx','LAy','LAz','GYx','GYy','GYz','name'])
for name in names:
    acc_named = acc_final[acc_final['name'] == name]
    gyro_named = gyro_final[gyro_final['name'] == name]
    acc_named = acc_named.drop('name', axis=1)
    merged_df = pd.merge(acc_named, gyro_named, on='timestamp', how='inner')
    merged_final = pd.concat([merged_final, merged_df], axis=0)

In [26]:
merged_final

,timestamp,LAx,LAy,LAz,GYx,GYy,GYz,name
0,924313410000000,1.242813,-0.160181,-0.900440,0.194931,-0.004242,0.163818,001
1,924313420000000,1.117246,-0.187921,-1.529893,0.236458,0.076378,0.133293,001
2,924313430000000,0.912846,-0.192566,-1.570929,0.226074,0.012253,0.108246,001
3,924313440000000,0.874858,-0.227208,-1.453511,0.139336,-0.050667,0.085030,001
4,924313450000000,0.834022,-0.282976,-1.292671,-0.013374,-0.109314,0.078308,001
...,...,...,...,...,...,...,...,...
29983,1368327120000000,-9.502644,-7.178468,4.246279,-2.834417,-0.120428,-0.987857,050
29984,1368327130000000,-9.443804,-7.698220,3.059675,-2.638940,0.561997,-0.947715,050
29985,1368327140000000,-8.345459,-6.805815,1.853457,-2.483606,1.225222,-0.954696,050
29986,1368327150000000,-7.580540,-5.629017,1.559257,-2.429500,1.769765,-1.003565,050


In [27]:
# gyro_final.to_csv("gyro_u1-50_w001_resampled.csv", index = False)
# acc_final.to_csv("acc_u1-50_w001_resampled.csv", index = False)


In [29]:
# merged_final.to_csv('merged_u1-50_w001_resampled.csv', index = False)